In [1]:
import numpy as np
import pandas as pd
#Import Retrosheet 2019 data as df19

In [54]:
df19.head()

,GAME_ID,AWAY_TEAM_ID,INN_CT,BAT_HOME_ID,OUTS_CT,BALLS_CT,STRIKES_CT,PITCH_SEQ_TX,AWAY_SCORE_CT,HOME_SCORE_CT,BAT_ID,BAT_HAND_CD,RESP_BAT_ID,RESP_BAT_HAND_CD,PIT_ID,PIT_HAND_CD,RESP_PIT_ID,RESP_PIT_HAND_CD,POS2_FLD_ID,POS3_FLD_ID,POS4_FLD_ID,POS5_FLD_ID,POS6_FLD_ID,POS7_FLD_ID,POS8_FLD_ID,POS9_FLD_ID,BASE1_RUN_ID,BASE2_RUN_ID,BASE3_RUN_ID,EVENT_TX,LEADOFF_FL,PH_FL,BAT_FLD_CD,BAT_LINEUP_ID,EVENT_CD,BAT_EVENT_FL,AB_FL,H_FL,SH_FL,SF_FL,EVENT_OUTS_CT,DP_FL,TP_FL,RBI_CT,WP_FL,PB_FL,FLD_CD,BATTEDBALL_CD,BUNT_FL,FOUL_FL,BATTEDBALL_LOC_TX,ERR_CT,ERR1_FLD_CD,ERR1_CD,ERR2_FLD_CD,ERR2_CD,ERR3_FLD_CD,ERR3_CD,BAT_DEST_ID,RUN1_DEST_ID,RUN2_DEST_ID,RUN3_DEST_ID,BAT_PLAY_TX,RUN1_PLAY_TX,RUN2_PLAY_TX,RUN3_PLAY_TX,RUN1_SB_FL,RUN2_SB_FL,RUN3_SB_FL,RUN1_CS_FL,RUN2_CS_FL,RUN3_CS_FL,RUN1_PK_FL,RUN2_PK_FL,RUN3_PK_FL,RUN1_RESP_PIT_ID,RUN2_RESP_PIT_ID,RUN3_RESP_PIT_ID,GAME_NEW_FL,GAME_END_FL,PR_RUN1_FL,PR_RUN2_FL,PR_RUN3_FL,REMOVED_FOR_PR_RUN1_ID,REMOVED_FOR_PR_RUN2_ID,REMOVED_FOR_PR_RUN3_ID,REMOVED_FOR_PH_BAT_ID,REMOVED_FOR_PH_BAT_FLD_CD,PO1_FLD_CD,PO2_FLD_CD,PO3_FLD_CD,ASS1_FLD_CD,ASS2_FLD_CD,ASS3_FLD_CD,ASS4_FLD_CD,ASS5_FLD_CD,EVENT_ID,RUNS,HALF.INNING,RUNS.SCORED
0,ANA201904040,TEX,1,0,0,2,2,FBBCH,0,0,choos001,L,choos001,L,harvm001,R,harvm001,R,lucrj001,bourj002,lastt001,cozaz001,simma001,goodb001,troum001,calhk001,NaN,NaN,NaN,HP,T,F,9,1,16,T,F,0,F,F,0,F,F,0,F,F,0,NaN,F,F,NaN,0,0,N,0,N,0,N,1,0,0,0,NaN,NaN,NaN,NaN,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,T,F,F,F,F,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,1,0,ANA201904040 1 0,0
1,ANA201904040,TEX,1,0,0,1,2,BFSS,0,0,odorr001,L,odorr001,L,harvm001,R,harvm001,R,lucrj001,bourj002,lastt001,cozaz001,simma001,goodb001,troum001,calhk001,choos001,NaN,NaN,K,F,F,4,2,3,T,T,0,F,F,1,F,F,0,F,F,0,NaN,F,F,NaN,0,0,N,0,N,0,N,0,1,0,0,2.0,NaN,NaN,NaN,F,F,F,F,F,F,F,F,F,harvm001,NaN,NaN,F,F,F,F,F,NaN,NaN,NaN,NaN,0,2,0,0,0,0,0,0,0,2,0,ANA201904040 1 0,0
2,ANA201904040,TEX,1,0,1,1,2,CCBX,0,0,andre001,R,andre001,R,harvm001,R,harvm001,R,lucrj001,bourj002,lastt001,cozaz001,simma001,goodb001,troum001,calhk001,choos001,NaN,NaN,S9/L.1-2,F,F,10,3,20,T,T,1,F,F,0,F,F,0,F,F,9,L,F,F,NaN,0,0,N,0,N,0,N,1,2,0,0,NaN,NaN,NaN,NaN,F,F,F,F,F,F,F,F,F,harvm001,NaN,NaN,F,F,F,F,F,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,3,0,ANA201904040 1 0,0
3,ANA201904040,TEX,1,0,1,0,0,X,0,0,gallj002,L,gallj002,L,harvm001,R,harvm001,R,lucrj001,bourj002,lastt001,cozaz001,simma001,goodb001,troum001,calhk001,andre001,choos001,NaN,HR/8/F.2-H;1-H,F,F,7,4,23,T,T,4,F,F,0,F,F,3,F,F,0,F,F,F,8,0,0,N,0,N,0,N,4,4,4,0,NaN,NaN,NaN,NaN,F,F,F,F,F,F,F,F,F,harvm001,harvm001,NaN,F,F,F,F,F,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,4,0,ANA201904040 1 0,3
4,ANA201904040,TEX,1,0,1,3,0,BBBB,3,0,cabra002,L,cabra002,L,harvm001,R,harvm001,R,lucrj001,bourj002,lastt001,cozaz001,simma001,goodb001,troum001,calhk001,NaN,NaN,NaN,W,F,F,5,5,14,T,F,0,F,F,0,F,F,0,F,F,0,NaN,F,F,NaN,0,0,N,0,N,0,N,1,0,0,0,NaN,NaN,NaN,NaN,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,F,F,F,F,F,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,5,3,ANA201904040 1 0,0


In [55]:
#REM for 2019 from Baseball Prospectus
REM = {'000 0':0.5439,'000 1':0.2983,'000 2':0.1147,'100 0':0.9345,'100 1':0.5641,'100 2':0.2422, \
       '010 0':1.1465,'010 1':0.7134,'010 2':0.3391,'110 0':1.5371,'110 1':0.9792,'110 2':0.4666, \
       '001 0':1.3685,'001 1':0.9528,'001 2':0.3907,'101 0':1.7591,'101 1':1.2186,'101 2':0.5182, \
       '011 0':1.9711,'011 1':1.3679,'011 2':0.6151,'111 0':2.3617,'111 1':1.6337,'111 2':0.7426}

In [56]:
#Create Half Inning, Runs scored on each play Vars
df19['RUNS'] = df19['AWAY_SCORE_CT'] + df19['HOME_SCORE_CT']
df19['HALF.INNING'] = df19['GAME_ID'].astype(str) + " " + df19['INN_CT'].astype(str) + " " + df19['BAT_HOME_ID'].astype(str)
df19['RUNS.SCORED'] = df19['BAT_DEST_ID'].apply(lambda x:1 if x>3 else 0)+\
    df19['RUN1_DEST_ID'].apply(lambda x:1 if x>3 else 0)+\
    df19['RUN2_DEST_ID'].apply(lambda x:1 if x>3 else 0)+\
    df19['RUN3_DEST_ID'].apply(lambda x:1 if x>3 else 0)


In [57]:
half_inn = pd.DataFrame()
half_inn['Outs.Inning'] = df19.groupby('HALF.INNING')['EVENT_OUTS_CT'].sum()
half_inn['Runs.Inning'] = df19.groupby('HALF.INNING')['RUNS.SCORED'].sum()
half_inn['Runs.Start'] = df19.groupby('HALF.INNING')['RUNS'].min()
half_inn['MAX.RUNS'] = half_inn['Runs.Inning'] + half_inn['Runs.Start']

In [58]:
df19 = pd.merge(df19,half_inn,how='inner',on='HALF.INNING')

In [59]:
#Add STATE
df19['BASE1'] = np.where(df19['BASE1_RUN_ID'].isnull(), 0, 1).astype(str)
df19['BASE2'] = np.where(df19['BASE2_RUN_ID'].isnull(), 0, 1).astype(str)
df19['BASE3'] = np.where(df19['BASE3_RUN_ID'].isnull(), 0, 1).astype(str)
df19['BASES'] = df19['BASE1'] + df19['BASE2'] + df19['BASE3']
df19['STATE'] = df19['BASES'] + " " + df19['OUTS_CT'].astype(str)
df19.drop(['BASE1','BASE2','BASE3'],axis=1,inplace=True)

In [60]:
#Add NEW.STATE
df19['NRUNNER1'] = np.where((df19['RUN1_DEST_ID'] == 1) | (df19['BAT_DEST_ID']==1), 1, 0).astype(str)
df19['NRUNNER2'] = np.where((df19['RUN1_DEST_ID'] == 2) | (df19['RUN2_DEST_ID']==2) | (df19['BAT_DEST_ID'] == 2), 1, 0).astype(str)
df19['NRUNNER3'] = np.where((df19['RUN1_DEST_ID'] == 3) | (df19['RUN2_DEST_ID'] == 3) | (df19['RUN3_DEST_ID'] == 3) | (df19['BAT_DEST_ID']==3), 1, 0).astype(str)
df19['NOUTS'] = df19['OUTS_CT'] + df19['EVENT_OUTS_CT']
df19['NEW.BASES'] = df19['NRUNNER1'] + df19['NRUNNER2'] + df19['NRUNNER3']
df19['NEW.STATE'] = df19['NEW.BASES'] + ' ' + df19['NOUTS'].astype(str)

In [61]:
#Limit to Run-Scoring or State-Changing Plays (where Run-Scoring doesn't happen on final out)
df19 = df19[((df19['STATE'] != df19['NEW.STATE']) | (df19['RUNS.SCORED'] > 0)) & (df19['Outs.Inning']==3) & (df19['BAT_EVENT_FL'] == 'T')]
df19['NEW.STATE'] = np.where(df19['NEW.STATE'].apply(lambda x:x.find("3"))>0, "3", df19['NEW.STATE'])

In [62]:
df19 = pd.merge(df19,pd.Series(REM).reset_index(),how='left',left_on='STATE',right_on='index')
df19 = pd.merge(df19,pd.Series(REM).reset_index(),how='left',left_on='NEW.STATE',right_on='index')

In [80]:
df19.drop(['index_x','index_y'],axis=1,inplace=True)
df19.rename(columns={'0_x':'Runs.State','0_y':'Runs.New.State'},inplace=True)

In [96]:
df19['Runs.New.State'] = df19['Runs.New.State'].fillna(value=0)
df19['Run.Value'] = df19['Runs.New.State'] - df19['Runs.State'] + df19['RUNS.SCORED']
df19['K Flag'] = np.where(df19['EVENT_CD']==3, 1, 0)
df19['BB Flag'] = np.where(df19['EVENT_CD']==14, 1, 0)
df19['BIP Flag'] = np.where(np.isin(df19['EVENT_CD'], [2,13,18,19,20,21,22,23]), 1, 0)
df19['Empty Flag'] = np.where(df19['STATE'].str.slice(stop=3) == '000', 1, 0)
df19['HR Flag'] = np.where(df19['EVENT_CD']==23, 1, 0)

In [93]:
#Mean RVs:
df19.groupby(['Empty Flag','K Flag'])['Run.Value'].mean()

Empty Flag  K Flag
0           0         0.110984
            1        -0.418464
1           0         0.052988
            1        -0.189675
Name: Run.Value, dtype: float64

In [94]:
df19.groupby(['Empty Flag','BB Flag'])['Run.Value'].mean()

Empty Flag  BB Flag
0           0         -0.038292
            1          0.370895
1           0         -0.029671
            1          0.279362
Name: Run.Value, dtype: float64

In [95]:
df19.groupby(['Empty Flag','BIP Flag'])['Run.Value'].mean()

Empty Flag  BIP Flag
0           0          -0.168312
            1           0.073131
1           0          -0.061553
            1           0.022697
Name: Run.Value, dtype: float64

In [97]:
df19.groupby(['Empty Flag','HR Flag'])['Run.Value'].mean()

Empty Flag  HR Flag
0           0         -0.074467
            1          1.922743
1           0         -0.043345
            1          1.000000
Name: Run.Value, dtype: float64

In [102]:
df19.groupby(['HR Flag'])['Run.Value'].mean()

HR Flag
0   -0.056640
1    1.382705
Name: Run.Value, dtype: float64